This notebook assumes we have already run a scrapy spider to collect race data like so:

```python
from processing.scrapers import LeadvilleScraper
LeadvilleScraper(2019).run_spider_pandas()
```

First, let's import the packages and define the variables that will be used throughout the data cleansing process.

In [1]:
import pandas as pd

from processing import cleaners, io, util

RACE_YEAR = 2019

## Split metadata

Before looking at the athlete splits, we must load, process, and clean the metadata about the splits themselves.

### Load raw metadata

Load the split info (as scraped by the spider) into a `pandas.DataFrame`.

In [2]:
df_split_info_raw = io.load_df_split_info_raw(RACE_YEAR)

df_split_info_raw

,distance_m
name,
Full Course,161578
12.6mi_Start to May Queen,20278
23.5mi_Start to Outward Bound OUT,37820
29.3mi_Start to Half Pipe Out,47154
37.9mi_Start to Twin Lakes Out,60994
43.5mi_Start to Hope Pass Out,70006
50mi_Start to Winfield,80467
56.5mi_Start to Hope Pass In,90928
62.5mi_Start to Twin Lakes In,100584


### Cleanse the metadata

First, sort the split info by ascending distance and add a column with mileage units:

In [3]:
df_split_info = cleaners.process_df_split_info(df_split_info_raw)

df_split_info

,distance_m,distance_mi
label,,
12.6mi_Start to May Queen,20278,12.6
23.5mi_Start to Outward Bound OUT,37820,23.5
29.3mi_Start to Half Pipe Out,47154,29.3
37.9mi_Start to Twin Lakes Out,60994,37.9
43.5mi_Start to Hope Pass Out,70006,43.5
50mi_Start to Winfield,80467,50.0
56.5mi_Start to Hope Pass In,90928,56.5
62.5mi_Start to Twin Lakes In,100584,62.5
71.1mi_Start to Half Pipe In,114424,71.1


Then, apply some Leadville-specific processing to the split info:

In [4]:
# (present in 2021, gone in 2022, and never very useful)
df_split_info.drop(index='Kick to Finish 0.8 Miles Left', errors='ignore', inplace=True)

# Manually add cutoff info (found elsewhere on the web)
df_split_info['cutoff_hr'] = [3.75, 6., 7.5, 9.5, 12., 14., None, 18., 21.25, 23., 26.5, 30.]

df_split_info

,distance_m,distance_mi,cutoff_hr
label,,,
12.6mi_Start to May Queen,20278,12.6,3.75
23.5mi_Start to Outward Bound OUT,37820,23.5,6.00
29.3mi_Start to Half Pipe Out,47154,29.3,7.50
37.9mi_Start to Twin Lakes Out,60994,37.9,9.50
43.5mi_Start to Hope Pass Out,70006,43.5,12.00
50mi_Start to Winfield,80467,50.0,14.00
56.5mi_Start to Hope Pass In,90928,56.5,NaN
62.5mi_Start to Twin Lakes In,100584,62.5,18.00
71.1mi_Start to Half Pipe In,114424,71.1,21.25


### Save cleaned metadata

Write the processed, cleaned split info DataFrame to a (`pandas`-preferred) CSV file for future use.

In [5]:
io.save_df_split_info_clean(df_split_info, RACE_YEAR)

## Athlete split data

The Athlinks spider saves split data for every athlete in one big `json` file.
We will load it into a single data structure, process it into the desired formats and types,
cleanse it by hunting for anomalous data, and finally save the cleansed data in a new file.

### Load data

Load all athlete split data from the spider-produced `json` file into a `pandas.DataFrame`:

In [6]:
df_split_times_raw = io.load_df_split_times_raw(RACE_YEAR)

util.df_td_fmt(df_split_times_raw)

,Lance Anderson,Clark Messman,Jacob Banta,Jeremy Bradford,Daniel Frank,Jeffrey Urbanski,Marvin Sandoval,Justin Ladner,Max Robinson,RUBEN DELGADO GIL,...,Gabe Joyes,Gediminas Grinius,Tate Knight,Mark Marzen,Jock Green,Chad Trammell,Jon Harrison,Ramon Casanovas,Devon Olson,Ryan Smith
Full Course,22:14:20,21:15:31,22:13:02,22:10:51,21:58:23,21:12:28,21:06:58,21:04:42,20:51:47,21:00:20,...,19:37:23,19:22:13,18:51:47,19:10:29,19:33:10,17:56:26,22:16:18,18:14:51,17:57:17,16:33:24
12.6mi_Start to May Queen,02:06:46,01:50:40,01:57:22,02:13:47,01:57:18,01:54:28,01:53:39,01:42:33,01:49:21,01:43:14,...,01:46:31,01:39:27,01:44:27,01:40:13,01:50:22,01:37:00,02:07:45,<NA>,01:36:25,01:36:52
23.5mi_Start to Outward Bound OUT,04:12:06,03:38:10,03:49:11,04:16:42,03:47:47,03:45:52,03:53:40,03:26:33,03:31:07,03:29:35,...,03:25:15,03:24:31,03:27:26,03:22:00,03:38:14,03:08:39,04:00:12,03:17:28,03:12:37,03:10:01
29.3mi_Start to Half Pipe Out,05:15:50,04:29:34,04:50:46,05:19:26,04:42:29,04:39:39,04:57:14,04:17:33,04:17:37,04:21:32,...,04:17:45,04:15:00,04:15:08,04:15:25,04:34:49,03:56:16,04:58:15,04:07:37,03:59:01,03:56:17
37.9mi_Start to Twin Lakes Out,06:57:08,05:54:20,06:30:59,06:58:14,06:16:30,06:10:57,06:34:37,05:42:44,05:41:57,05:53:35,...,05:57:26,05:42:35,05:36:24,05:41:39,06:07:03,05:13:27,06:42:05,05:37:23,05:18:26,05:14:23
43.5mi_Start to Hope Pass Out,08:59:45,07:49:04,08:23:18,08:53:07,08:17:31,08:00:26,08:30:20,07:33:02,07:20:59,07:49:31,...,07:38:26,07:30:55,07:10:54,07:33:21,07:52:38,06:42:38,08:47:10,07:14:04,07:00:41,06:46:09
50mi_Start to Winfield,10:17:04,09:10:43,09:50:52,10:07:13,09:36:05,09:19:39,09:45:56,08:50:00,08:36:49,09:04:26,...,08:42:49,08:40:04,08:16:57,08:38:57,09:04:44,07:44:21,10:12:04,08:18:15,08:04:28,07:47:15
56.5mi_Start to Hope Pass In,12:24:52,11:28:39,11:55:39,12:06:26,11:41:16,11:21:07,11:32:39,10:48:53,10:42:53,11:09:08,...,10:27:19,10:35:46,10:04:32,10:36:10,10:46:44,09:35:10,12:19:28,09:56:02,09:55:50,09:22:44
62.5mi_Start to Twin Lakes In,13:46:41,13:04:04,13:38:10,13:25:09,12:57:45,12:46:14,12:46:23,12:11:28,12:11:25,12:27:43,...,11:39:48,11:40:18,11:08:58,11:33:27,11:50:38,10:37:24,13:45:52,11:02:04,11:00:26,10:17:55
71.1mi_Start to Half Pipe In,15:41:10,14:40:29,15:33:27,15:15:33,14:49:31,14:39:38,14:36:59,14:01:03,14:00:47,14:16:41,...,13:21:05,13:23:23,12:49:03,13:13:09,13:31:37,12:13:37,15:40:52,12:42:01,12:35:46,11:49:43


### Cleanse data

We can inspect the whole DataFrame for abnormalities. We are mostly looking for anomalous
finish line splits when a runner DNFs and returns to Leadville, but there are
occasional split misfires mid-race too.

Notice that in the DataFrame above, the columns (athletes) are presented in the order chosen by 
the Athlinks website spider. This isn't particularly useful for spotting outliers or unreasonable
performances. It would be easier to spot bad data if athletes with similar performances were 
grouped together.

To fix this, we can group the athletes based on the furthest split they each recorded, then sort
athletes within each group by their split times:

In [7]:
df_split_times = cleaners.sort_df_split_data(df_split_times_raw, df_split_info)

util.display_full_df(util.df_td_fmt(df_split_times))

Charles Corfield Ryan Smith Chad Trammell Devon Olson Ramon Casanovas Tate Knight Mark Marzen Gediminas Grinius Jock Green Gabe Joyes Matthew Urbanski Magdalena Boulet Vineer Bhansali Christopher DeNucci Rodrigo Jimenez Cat Bradley Max Robinson RUBEN DELGADO GIL Justin Ladner Marvin Sandoval Jeffrey Urbanski Clark Messman Radu Milea steve slauson Daniel Frank Anton Samokhvalov Jeremy Bradford Jacob Banta Lance Anderson Jon Harrison Peter Mortimer  Pat Cade Matthew Harrison Wesley Sandoval Mackenzie Mau Jamie Kilcoyne David Goggins Ryan Van Ness Chad Braun Jonathan Lantz TODD DILL Bryan Kerl Adam Monke Nathaniel Orders Tito Nazar Ezra Becker Zachary Allen Kilian Korth Brooks Williams John Olar Brandon Worthington Samantha Wood Carrie Stafford James Schneiderman Bruce Dailey Dustin Moore Jordan Ricks Harry Harcrow Jared Conlin Timothy Barr Willie Stewart Chavet Breslin Tim Finocchio Sacha Devillaz Phil Reutlinger Drew Adams John Muir Max Fulton paul zani Lelis Gonzalez Brandon Ray Steve Orellana Jens Sperlich Alex Cummings matt harvey Steve Pentler Howard Vogt Daniel Arnold Sam Schwaller Aaron Acuna Leonardo Leite Tim Cronk Michael Robbert Robert Stowe Travis Daniels Niki Pardoe ANDREW WICKLESS jason vieth Michael Alfred Kaitlyn Morgan George Ulrich Michael King Dion Leonard Aaron Ellison Joshua Stevens Craig Sylvester Shannon Howell Chris Wright kenneth ebener RANDY MAST Shaun Baker Gary Stotler aaron fearing Nikita Levchenko Kara Diamond-Husmann David Mackey Jonathan Joslyn Paul Tucker Mat Grills Stacey Marion Peter Grenney Jason Romero Francesc Basacoma Matthew Hunsucker Janis Snepste Ondrej Tomek Richie Dart Neil Feldman Benjamin Hauschulz Chris Mead Rod Bolls Emily Messegee Joshua Dickson Hannah Jochem Katrin Silva MELANIE WHITE kenneth erichsen Stephen Rodgers Giles Hedley John Mollenhour Daniela Seyler Walter Handloser Bill Hoffman Alan Smith Cristobal Forno David Kerr III Clayton Shapiro Dan Zolnikov Christine O'Gorman Jeremy Howard Hope Bain Tracy O'Brien Anders Wood John Goble Randy Semrau Jay Hagan Emily Luhrs Cristian Vargas Max Lawler Nic De Beer Jeff Mohrmann Sean McLaughlin Rob Wright Katy Anderson Amar Kuchinad Cory Leppert Matthew Kirkendall Dreama Walton Wade Jarvis Max Forgensi Tom Welker Andrew Flinn Drew McManus Andy Bourke Jeff Zenger Martin Schneekloth R. Sean Churchill Michael Nothem Michael Sylvester Gabriel Hendry Erin Park Lillie Romeiser Zach Allen Lucas Onan Andrew Coleman Luke Pospisil Eric Nelson Robert Wright Brandon Wilmoth Denise McHale javier vilchis Clark Mitchell Caitlin Barr Neal Palles April Kenyon Marc Grabowski Jemil Kassahun Kenneth Biddison Erin Drasler John (Muck) kilpatrick TR Maloney Carey Willson LaMar Jost John Paul Ogden Lance Pearce Pete Kardasis Phil Atkinson Ted Richardson John Music Sonja Brester Robert J Koch Mike Wasson Geoff Hanley Canice Harte BAREFOOT MCDONALD Scott Sundahl James Koch Brandy Taylor Jason Goss Joe Miller Paul Dube Newton McCollum tyler quinn Kayde Anderson Lisa Nelson Andrew Erickson Mihai Serban ROBERT MAZURKIEWICZ helen degennaro Caspar Sprungli Brian Burton Ryan McAfee Trey Simcik Becky Grebosky Donna Provenzano Denzil Jennings Alexandra Mittelman Jeremiah Jenson Sam Stephenson Bruno Furrer Amanda Webb Neil Smith Colleen Mentz Chris Kumm Jason Carey Brad Anderson Ciro Ortega Andrew Gzybowski Madeline Hirschfeld Jeremiah Nemechek Erik Nielsen Moon-Ka LEUNG Jack Repasky Erik Hull Brian Merseal Dave Ballard Kevin Moore Erin Laplander Charles Bybee Walter Edwards Kevin Koncilja Jimmy Daly Brian Passenti Abby Reinholt Evan Weisel Junko Kazukawa Bryan Wisdom Allen Kinsler Trish Holbel Sergio Saenz Robb McLean RAMIRO HECTOR CALDERON  COLOMO Paul Nielsen Chung-Man Tam Tyler Patterson Ted Politte Ethan Pence Todd Pizzi Sean Doran Monica Obsitos Jacey Anderson Karolina Zavisiute Jim Ploof Jack Rawls Aaron Cook Benjamin Corwin Luke Finken Alexa Roop Stephen Brown Brent Heady Kristin Walls Daniel Madalena Hawaiian Shirt Ray Churgovich Nathan Pierce Ben Santerre Har

#### Optional: Print entire splits DataFrame to a file

If desired, we can save this file and scroll through it in our preferred
text editor. Some notes:

* VSCode (Windows): alt+z to disable wrap.
* VSCode will not display files in excess of 10k characters in width.
  Sublime text can do it.

```python
util.full_df_to_file(
  util.df_td_fmt(df_split_times),
  os.path.join(io.get_raw_race_data_dir(RACE_YEAR), 'df_all.txt')
)
```

#### Remove bad data

Next we will remove some obvious wonky runner data, as determined by manual inspection.

Sorting the DataFrame highlights erroneous split times. For example, look at the first column, containing the splits for Charles Corfield. The uncleaned split data suggests that Charles won the entire race, because his finish line split is the fastest. But looking more closely, it seems he dropped out of the race after the split labeled `56.5mi_Start to Hope Pass In`, and his bib triggered the sensor at the finish line when he came to watch the race winner, Ryan Smith, bring it home. Something similar seemed to happen with Vineer Bhansali. To see what I mean, compare their earlier, valid splits to Ryan's:

In [8]:
util.df_td_fmt(
  df_split_times[['Ryan Smith', 'Charles Corfield', 'Vineer Bhansali']]
)

,Ryan Smith,Charles Corfield,Vineer Bhansali
label,,,
12.6mi_Start to May Queen,01:36:52,02:25:16,02:32:48
23.5mi_Start to Outward Bound OUT,03:10:01,04:37:43,05:15:49
29.3mi_Start to Half Pipe Out,03:56:17,05:44:52,06:38:03
37.9mi_Start to Twin Lakes Out,05:14:23,07:35:51,08:52:44
43.5mi_Start to Hope Pass Out,06:46:09,10:05:19,12:03:43
50mi_Start to Winfield,07:47:15,11:35:04,13:58:44
56.5mi_Start to Hope Pass In,09:22:44,13:56:55,<NA>
62.5mi_Start to Twin Lakes In,10:17:55,<NA>,<NA>
71.1mi_Start to Half Pipe In,11:49:43,<NA>,<NA>


To cleanse the data of these erroneous splits, we just need to set them to a null value.

In [9]:
df_split_times.loc['Full Course', 'Charles Corfield'] = pd.NA
df_split_times.loc['Full Course', 'Vineer Bhansali'] = pd.NA

util.df_td_fmt(
  df_split_times[['Charles Corfield', 'Vineer Bhansali']]
)

,Charles Corfield,Vineer Bhansali
label,,
12.6mi_Start to May Queen,02:25:16,02:32:48
23.5mi_Start to Outward Bound OUT,04:37:43,05:15:49
29.3mi_Start to Half Pipe Out,05:44:52,06:38:03
37.9mi_Start to Twin Lakes Out,07:35:51,08:52:44
43.5mi_Start to Hope Pass Out,10:05:19,12:03:43
50mi_Start to Winfield,11:35:04,13:58:44
56.5mi_Start to Hope Pass In,13:56:55,<NA>
62.5mi_Start to Twin Lakes In,<NA>,<NA>
71.1mi_Start to Half Pipe In,<NA>,<NA>


Another type of error can happen when a split is triggered erroneously early, for whatever reason. This type of erroneous data doesn't necessarily jump out upon visual inspection. One way to check for it is to see if any athletes data includes a negative point-to-point time.

In [10]:
util.df_td_fmt(
  util.get_df_negative_timedelta(df_split_times)
)

,Willie Stewart,Max Fulton
label,,
12.6mi_Start to May Queen,02:37:41,02:07:35
23.5mi_Start to Outward Bound OUT,05:05:51,<NA>
29.3mi_Start to Half Pipe Out,06:25:48,05:19:12
37.9mi_Start to Twin Lakes Out,08:34:21,<NA>
43.5mi_Start to Hope Pass Out,10:52:02,12:51:24
50mi_Start to Winfield,12:36:47,10:34:47
56.5mi_Start to Hope Pass In,15:13:10,<NA>
62.5mi_Start to Twin Lakes In,17:25:27,14:26:52
71.1mi_Start to Half Pipe In,20:55:16,<NA>


It's not immediately clear what is going on with the split data for either of these athletes.

In Willie's case, the erroneously negative elapsed time comes between the splits at 76.9 miles and 87.8 miles. But even if we discarded the split data at 87.8 miles as an obvious misfire, no one could cover the distance from 76.9 miles to the finish in such a short time. But is the error at 76.9 miles or the finish line? Both?

It's impossible to tell which split data is valid or invalid without more context. Let's look at athletes with similar finish times to each athlete with bad data.

In [11]:
util.df_td_fmt(
  util.get_column_context(df_split_times, 'Willie Stewart')
)

,Dustin Moore,Jordan Ricks,Harry Harcrow,Jared Conlin,Timothy Barr,Willie Stewart,Chavet Breslin,Tim Finocchio,Sacha Devillaz,Phil Reutlinger
label,,,,,,,,,,
12.6mi_Start to May Queen,01:54:36,01:53:51,02:10:23,02:10:16,01:57:09,02:37:41,02:27:43,02:11:46,01:48:54,01:45:33
23.5mi_Start to Outward Bound OUT,03:46:50,03:50:14,04:25:07,04:23:24,03:58:18,05:05:51,05:01:15,04:26:39,03:42:10,03:27:17
29.3mi_Start to Half Pipe Out,04:41:33,04:49:37,05:36:59,05:33:02,05:03:23,06:25:48,06:15:07,05:33:07,04:44:40,04:18:51
37.9mi_Start to Twin Lakes Out,06:08:45,06:34:31,07:30:58,07:25:00,06:48:51,08:34:21,08:18:11,07:29:20,06:36:00,05:49:22
43.5mi_Start to Hope Pass Out,07:58:00,08:39:24,10:05:20,09:55:27,08:53:27,10:52:02,11:18:24,09:38:59,08:40:43,07:45:10
50mi_Start to Winfield,09:10:02,10:03:20,11:32:32,11:28:55,10:15:48,12:36:47,13:29:12,11:10:27,10:06:14,09:07:02
56.5mi_Start to Hope Pass In,11:09:45,12:10:13,14:15:05,13:55:57,12:21:24,15:13:10,15:59:55,13:32:45,12:24:45,11:23:10
62.5mi_Start to Twin Lakes In,12:21:07,13:43:02,16:35:13,15:49:43,13:52:13,17:25:27,<NA>,15:01:03,13:46:35,13:10:05
71.1mi_Start to Half Pipe In,14:14:20,15:42:24,19:17:30,19:16:10,15:49:39,20:55:16,21:18:09,17:04:44,15:50:11,15:18:17


With the added context of surrounding runners, it becomes clear that there are more data issues.
It's kind of hard to definitively judge which splits are valid and which are bogus.

* It seems likely that Willie Stewart DNFed at 76.9. I just doubt anyone could cover 71.1 -> 100 in 3ish hours,
  so it seems very unlikely that the split at 76.9 is a misfire.
* Chavet Breslin probably DNFed at 76.9 and recorded an erroneous finish.
* Harry Harcrow and Jared Conlin seem to have DNFed at 71.1 but recorded a finish line split near each other.
* Tim Finocchio seemed sketchy at first with how much ground he made up in the later splits, but I
  cannot say anything without busting out the statistics.

Let's cleanse the more likely errors:

In [12]:
df_split_times.loc['87.8mi_Start to May Queen In': 'Full Course', 'Willie Stewart'] = pd.NA
df_split_times.loc['Full Course', 'Chavet Breslin'] = pd.NA
df_split_times.loc['Full Course', 'Harry Harcrow'] = pd.NA
df_split_times.loc['Full Course', 'Jared Conlin'] = pd.NA

Next, let's look at the runners coming in around Max's recorded finish time:

In [13]:
util.df_td_fmt(
  util.get_column_context(df_split_times, 'Max Fulton')
)

,Tim Finocchio,Sacha Devillaz,Phil Reutlinger,Drew Adams,John Muir,Max Fulton,paul zani,Lelis Gonzalez,Brandon Ray,Steve Orellana
label,,,,,,,,,,
12.6mi_Start to May Queen,02:11:46,01:48:54,01:45:33,02:05:26,01:57:09,02:07:35,02:13:43,01:55:02,01:58:26,01:57:08
23.5mi_Start to Outward Bound OUT,04:26:39,03:42:10,03:27:17,04:09:53,03:52:19,<NA>,<NA>,03:48:48,<NA>,03:50:59
29.3mi_Start to Half Pipe Out,05:33:07,04:44:40,04:18:51,05:13:16,04:52:39,05:19:12,05:20:37,04:45:57,05:09:08,04:56:55
37.9mi_Start to Twin Lakes Out,07:29:20,06:36:00,05:49:22,07:06:31,06:38:57,<NA>,07:04:58,06:27:16,06:59:22,06:37:55
43.5mi_Start to Hope Pass Out,09:38:59,08:40:43,07:45:10,09:14:54,08:49:31,12:51:24,09:07:22,08:22:41,09:18:31,08:53:13
50mi_Start to Winfield,11:10:27,10:06:14,09:07:02,10:45:13,10:12:16,10:34:47,10:50:19,09:54:44,10:45:07,10:25:28
56.5mi_Start to Hope Pass In,13:32:45,12:24:45,11:23:10,13:12:36,12:33:15,<NA>,13:02:51,11:53:52,12:59:48,12:52:42
62.5mi_Start to Twin Lakes In,15:01:03,13:46:35,13:10:05,14:43:07,14:05:07,14:26:52,14:45:50,13:36:44,14:29:31,14:25:16
71.1mi_Start to Half Pipe In,17:04:44,15:50:11,15:18:17,16:51:55,16:14:47,<NA>,16:47:20,15:51:56,16:33:59,16:33:57


Immediately, it looks like the split at 43.5 misfired. I think his chip didn't fire on the way out,
but it did fire on the way in. Let's swap those values.

In [14]:
df_split_times.loc['56.5mi_Start to Hope Pass In', 'Max Fulton'] = df_split_times.loc[
                   '43.5mi_Start to Hope Pass Out', 'Max Fulton']
df_split_times.loc['43.5mi_Start to Hope Pass Out', 'Max Fulton'] = pd.NA

Re-sort the splits DataFrame now that some cellular data has changed.

In [15]:
df_split_times = cleaners.sort_df_split_data(df_split_times, df_split_info)

util.display_full_df(util.df_td_fmt(df_split_times))

# Debug
# util.df_td_fmt(
#   util.get_df_negative_timedelta(df_split_info)
# )

Ryan Smith Chad Trammell Devon Olson Ramon Casanovas Tate Knight Mark Marzen Gediminas Grinius Jock Green Gabe Joyes Matthew Urbanski Magdalena Boulet Christopher DeNucci Rodrigo Jimenez Cat Bradley Max Robinson RUBEN DELGADO GIL Justin Ladner Marvin Sandoval Jeffrey Urbanski Clark Messman Radu Milea steve slauson Daniel Frank Anton Samokhvalov Jeremy Bradford Jacob Banta Lance Anderson Jon Harrison Peter Mortimer  Pat Cade Matthew Harrison Wesley Sandoval Mackenzie Mau Jamie Kilcoyne David Goggins Ryan Van Ness Chad Braun Jonathan Lantz TODD DILL Bryan Kerl Adam Monke Nathaniel Orders Tito Nazar Ezra Becker Zachary Allen Kilian Korth Brooks Williams John Olar Brandon Worthington Samantha Wood Carrie Stafford James Schneiderman Bruce Dailey Dustin Moore Jordan Ricks Timothy Barr Tim Finocchio Sacha Devillaz Phil Reutlinger Drew Adams John Muir Max Fulton paul zani Lelis Gonzalez Brandon Ray Steve Orellana Jens Sperlich Alex Cummings matt harvey Steve Pentler Howard Vogt Daniel Arnold Sam Schwaller Aaron Acuna Leonardo Leite Tim Cronk Michael Robbert Robert Stowe Travis Daniels Niki Pardoe ANDREW WICKLESS jason vieth Michael Alfred Kaitlyn Morgan George Ulrich Michael King Dion Leonard Aaron Ellison Joshua Stevens Craig Sylvester Shannon Howell Chris Wright kenneth ebener RANDY MAST Shaun Baker Gary Stotler aaron fearing Nikita Levchenko Kara Diamond-Husmann David Mackey Jonathan Joslyn Paul Tucker Mat Grills Stacey Marion Peter Grenney Jason Romero Francesc Basacoma Matthew Hunsucker Janis Snepste Ondrej Tomek Richie Dart Neil Feldman Benjamin Hauschulz Chris Mead Rod Bolls Emily Messegee Joshua Dickson Hannah Jochem Katrin Silva MELANIE WHITE kenneth erichsen Stephen Rodgers Giles Hedley John Mollenhour Daniela Seyler Walter Handloser Bill Hoffman Alan Smith Cristobal Forno David Kerr III Clayton Shapiro Dan Zolnikov Christine O'Gorman Jeremy Howard Hope Bain Tracy O'Brien Anders Wood John Goble Randy Semrau Jay Hagan Emily Luhrs Cristian Vargas Max Lawler Nic De Beer Jeff Mohrmann Sean McLaughlin Rob Wright Katy Anderson Amar Kuchinad Cory Leppert Matthew Kirkendall Dreama Walton Wade Jarvis Max Forgensi Tom Welker Andrew Flinn Drew McManus Andy Bourke Jeff Zenger Martin Schneekloth R. Sean Churchill Michael Nothem Michael Sylvester Gabriel Hendry Erin Park Lillie Romeiser Zach Allen Lucas Onan Andrew Coleman Luke Pospisil Eric Nelson Robert Wright Brandon Wilmoth Denise McHale javier vilchis Clark Mitchell Caitlin Barr Neal Palles April Kenyon Marc Grabowski Jemil Kassahun Kenneth Biddison Erin Drasler John (Muck) kilpatrick TR Maloney Carey Willson LaMar Jost John Paul Ogden Lance Pearce Pete Kardasis Phil Atkinson Ted Richardson John Music Sonja Brester Robert J Koch Mike Wasson Geoff Hanley Canice Harte BAREFOOT MCDONALD Scott Sundahl James Koch Brandy Taylor Jason Goss Joe Miller Paul Dube Newton McCollum tyler quinn Kayde Anderson Lisa Nelson Andrew Erickson Mihai Serban ROBERT MAZURKIEWICZ helen degennaro Caspar Sprungli Brian Burton Ryan McAfee Trey Simcik Becky Grebosky Donna Provenzano Denzil Jennings Alexandra Mittelman Jeremiah Jenson Sam Stephenson Bruno Furrer Amanda Webb Neil Smith Colleen Mentz Chris Kumm Jason Carey Brad Anderson Ciro Ortega Andrew Gzybowski Madeline Hirschfeld Jeremiah Nemechek Erik Nielsen Moon-Ka LEUNG Jack Repasky Erik Hull Brian Merseal Dave Ballard Kevin Moore Erin Laplander Charles Bybee Walter Edwards Kevin Koncilja Jimmy Daly Brian Passenti Abby Reinholt Evan Weisel Junko Kazukawa Bryan Wisdom Allen Kinsler Trish Holbel Sergio Saenz Robb McLean RAMIRO HECTOR CALDERON  COLOMO Paul Nielsen Chung-Man Tam Tyler Patterson Ted Politte Ethan Pence Todd Pizzi Sean Doran Monica Obsitos Jacey Anderson Karolina Zavisiute Jim Ploof Jack Rawls Aaron Cook Benjamin Corwin Luke Finken Alexa Roop Stephen Brown Brent Heady Kristin Walls Daniel Madalena Hawaiian Shirt Ray Churgovich Nathan Pierce Ben Santerre Harrison Hornung Lexi Miller Chris Klinghagen Chris Minton Jason Steinberg mike yasinski Case

In the future, I'd like to use statistical methods and reasonable assumptions to 
programmatically flag sketchy split times. Single splits that are
anomalously fast are invisible to me when I'm visually checking the
output. 

Creating and reviewing plots of the data can help spot obvious outliers, but still,
sketchy splits may or may not stand out in a single scatterplot.

Implementing a more rigorous anomaly detection protocol sounds fun.

### Save the cleaned data

Write the processed, cleaned athlete split DataFrame to a (`pandas`-preferred) CSV file for future analysis.

In [16]:
io.save_df_split_times_clean(df_split_times, RACE_YEAR)